# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 4 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.

### Preamble

In [ ]:
import numpy as np
import qutip as qt

### Exercise 4.1 

**Since the referee is purely classical, we'll model them as using classical random number generators.
This leaves open the possibility, though, that you and Eve could cheat by guessing the referee's questions.
A possible improvement might be to use the QRNGs from Chapter 2.
Modify the code sample in [`chsh.py`](./chsh.py) so that the referee can ask questions of you and Eve by measuring a qubit that starts off in the $\left|+\right\rangle$ state.**

The key to this exercise is replacing `random_bit`:

```python
def random_bit() -> int:
    return random.randint(0, 1)
```

You can use what you learned in Chapter 2 to replace this by a QRNG:

```python
def random_bit(device: QuantumDevice) -> int:
    with device.using_qubit() as q:
        q.h()
        return int(q.measure())
```

The referee will then need to allocate a new quantum device to use in generating their random bits:

```python
def referee(strategy: Callable[[], Strategy]) -> bool:
    device = Simulator(capacity=1)
    you, eve = strategy()
    your_input, eve_input = random_bit(device), random_bit(device)
    # ...
```

----
### Exercise 4.2

**How would you prepare a $\left|+0\right\rangle$ state?
First, what vector would you use to represent the two-qubit state $\left|+0\right\rangle = \left|+\right\rangle \otimes \left|0\right\rangle$?
You have an initial two qubit register in the $\left|00\right\rangle$ state.
What operation should you apply to get the state you want?**

*HINT*: try (𝐻 ⊗ 𝟙) if you are stuck!

For this exercise, you can start by writing out the vector representations of $|+\rangle$ and $|0\rangle$ as QuTiP `Qobj` instances:

In [ ]:
ket0 = qt.Qobj([
    [1],
    [0]
])
ket_plus = qt.Qobj([
    [1],
    [1]
]) / np.sqrt(2)

You can then find the two-qubit state $|+0\rangle = |{+}\rangle \otimes |0\rangle$ using `qt.tensor`:

In [ ]:
qt.tensor(ket_plus, ket0)

Using the hint above, you can verify that this is what you get if you apply the Hadamard operation (represented by the unitary matrix returned by `qt.qip.operations.hadamard_transform`) to the first qubit of a two-qubit register. Note that as mentioned in the hint, you'll need to act on the state of the second qubit with identity matrix to represent doing nothing. The identity matrix can be obtained using the QuTiP function `qt.qeye(2)`.

In [ ]:
U = qt.tensor(qt.qip.operations.hadamard_transform(), qt.qeye(2))

You can then verify that applying the Hadamard operation to the first qubit does what you expect to the state of the two-qubit register.

In [ ]:
U * qt.tensor(ket0, ket0)

Since this agrees with the output of `qt.tensor(ket_plus, ket0)`, you've confirmed that you can use the Hadamard operation on the first qubit in a two-qubit register to prepare $|+0\rangle = |+\rangle \otimes |0\rangle$